In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, classification_report, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
warnings.filterwarnings("ignore")
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
rb=RobustScaler()
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
from imblearn.over_sampling import SMOTE

In [ ]:
dataset = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
# dataset.info()
dataset.isnull().sum()
dataset.bmi.replace(to_replace=np.nan, value=dataset.bmi.mean(), inplace=True)
dataset.isnull().sum()
dataset.head(10)


In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.info()

In [ ]:
corr = dataset.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap='YlGnBu', vmax=.4, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# Convert Marrital Status, Residence and Gender into 0's and 1's
dataset['gender']=dataset['gender'].apply(lambda x : 1 if x=='Male' else 0)
dataset["Residence_type"] = dataset["Residence_type"].apply(lambda x: 1 if x=="Urban" else 0)
dataset["ever_married"] = dataset["ever_married"].apply(lambda x: 1 if x=="Yes" else 0)
# Removing the observations that have smoking type unknown.
dataset=dataset[dataset['smoking_status']!='Unknown']
# One Hot encoding smoking_status, work_type
data_dummies = dataset[['smoking_status','work_type']]
data_dummies=pd.get_dummies(data_dummies)
dataset.drop(columns=['smoking_status','work_type'],inplace=True)
data_stroke=dataset['stroke']
dataset.drop(columns=['stroke'],inplace=True)
dataset=dataset.merge(data_dummies,left_index=True, right_index=True,how='left')


In [ ]:
x = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [0,5,9])], remainder= 'passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 15] = le.fit_transform(x[:, 15])
x[:, 16] = le.fit_transform(x[:, 16])
#Splitting the dataset into the train set and testset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 0)
print(" x_train dataset: ", x_train.shape)
print("y_train dataset: ", y_train.shape)
print("x_test dataset: ", x_test.shape)
print(" y_test dataset: ", y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
#Feature Scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
#Imbalance data using SMOTE 
sm = SMOTE(random_state=0)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

In [ ]:
models = []

models.append(['MLP', MLPClassifier(random_state=50)])
models.append(['Logistic Regreesion', LogisticRegression(random_state=50)])
models.append(['SVM', SVC(random_state=0)])
models.append(['LDA',LinearDiscriminantAnalysis(solver='svd')])
models.append(['KNeighbors', KNeighborsClassifier()])
models.append(['Decision Tree', DecisionTreeClassifier(random_state=50)])
models.append(['Random Forest', RandomForestClassifier(random_state=50)])

lst_1= []

for m in range(len(models)):
    lst_2= []
    model = models[m][1]
    model.fit(x_train_res, y_train_res)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize = (8, 5))
    sns.heatmap(cm, cmap = 'YlGnBu', annot = True, fmt = 'd', linewidths = 4, cbar = True, annot_kws = {'fontsize': 15}, 
    yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
    plt.yticks(rotation = 0)
    plt.show()
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 5) 
    print(models[m][0],':')
    print(accuracies)
    print('Accuracy: ',accuracy_score(y_test, y_pred))
    print("k-fold mean acc: {:.4f} %".format(accuracies.mean()*100))
    lst_2.append(models[m][0])
    lst_2.append((accuracy_score(y_test, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_1.append(lst_2)

In [ ]:
dataset = pd.DataFrame(lst_1, columns= ['Model', 'Accuracy', 'K-Fold Mean Accuracy'])
dataset.sort_values(by= ['Accuracy', 'K-Fold Mean Accuracy'], inplace= True, ascending= False)
dataset

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gr_model = [(MLPClassifier(),  [
{'hidden_layer_sizes': [(100,1), (100,2), (100,3)],
'activation': ["logistic", "relu", "Tanh"]}]), 
               (KNeighborsClassifier(),[{'n_neighbors':[5,7,8,10], 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}]), 
               (DecisionTreeClassifier(),[{'criterion':['gini','entropy'],'random_state':[0]}]), 
               (RandomForestClassifier(),[{'n_estimators':[100,150,200],'criterion':['gini','entropy'],'random_state':[0]}]), 
              ]

for i,j in gr_model:
    grid = GridSearchCV(estimator=i,param_grid = j, scoring = 'accuracy',cv = 5)
    grid.fit(x_train_res, y_train_res)
    best_accuracy = grid.best_score_
    best_param = grid.best_params_
    print('{}:\n accuracy : {:.4f}%'.format(i,best_accuracy*100))
    print('params : ',best_param)

    

In [ ]:
models = []

models.append(['MLP', MLPClassifier(random_state=50,hidden_layer_sizes=(100, 2),activation="relu")])
models.append(['KNeighbors', KNeighborsClassifier(metric='manhattan',n_neighbors=5)])
models.append(['Decision Tree', DecisionTreeClassifier(criterion='entropy',random_state= 0)])
models.append(['Random Forest', RandomForestClassifier(n_estimators=100,random_state=0,criterion='gini')])

lst_1= []

for m in range(len(models)):
    lst_2= []
    model = models[m][1]
    model.fit(x_train_res, y_train_res)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize = (8, 4))
    sns.heatmap(cm, cmap = 'YlGnBu', annot = True, fmt = 'd', linewidths = 4, cbar = True, annot_kws = {'fontsize': 14}, 
    yticklabels = ['no stroke', 'stroke'], xticklabels = ['Pred_no stroke', 'Pred_stroke'])
    plt.yticks(rotation = 0)
    plt.show()
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 5)   #K-Fold Validation
    print(models[m][0],':')
    print(accuracies)
    print('Accuracy: ',accuracy_score(y_test, y_pred))
    print("K-Foldcv mean accuracy: {:.4f} %".format(accuracies.mean()*100))
    lst_2.append(models[m][0])
    lst_2.append((accuracy_score(y_test, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_1.append(lst_2)